In [ ]:
import torch
import ptychi.api as api
from ptychi.api.task import PtychographyTask
from ptychi.utils import get_default_complex_dtype
import matplotlib.pyplot as plt
import os
import numpy as np
import cupy as cp
import dxchange
import sys


In [ ]:
n = 1024  # data size in each dimension
nobj = n+n//4 # object size in each dimension
pad = 0 # pad for the reconstructed probe
nprb = n+2*pad # probe size
extra = 8 # extra padding for shifts
npatch = nprb+2*extra # patch size for shifts

npos = 16 # total number of positions
z1 = 4.267e-3 # [m] position of the sample
detector_pixelsize = 3.0e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
# adjustments for the cone beam
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = cp.abs(detector_pixelsize/magnification)

# reconstructon parameters
method = 'epie'
niter = 32 # number of iterations
eps = 1e-8 # to avoid division by 0
noise = False # noisy data

err_step = 1 # iteration step to calculate the minimization functional
vis_step = -1 # iteration step to visualize reconstructions
show = True # do visualization or not at all
flg = f'{method}_{noise}' # flg to save data

path = f'/data/vnikitin/paper/near_field' # input data path 
path_out = f'/data/vnikitin/paper/near_field/rec0' #output data path

alpha_psi = 0.1
alpha_prb = 0.1

In [ ]:
n = 1024  # data size in each dimension
nobj = n+n//4 # object size in each dimension
pad = 0 # pad for the reconstructed probe
nprb = n+2*pad # probe size
extra = 8 # extra padding for shifts
npatch = nprb+2*extra # patch size for shifts

npos = 16 # total number of positions
z1 = 4.267e-3 # [m] position of the sample
detector_pixelsize = 3.0e-6
energy = 33.35  # [keV] xray energy
wavelength = 1.24e-09/energy  # [m] wave length
focusToDetectorDistance = 1.28  # [m]
# adjustments for the cone beam
z2 = focusToDetectorDistance-z1
distance = (z1*z2)/focusToDetectorDistance
magnification = focusToDetectorDistance/z1
voxelsize = cp.abs(detector_pixelsize/magnification)

# reconstructon parameters
method = 'epie'
niter = 32 # number of iterations
eps = 1e-8 # to avoid division by 0
noise = False # noisy data

err_step = 1 # iteration step to calculate the minimization functional
vis_step = -1 # iteration step to visualize reconstructions
show = True # do visualization or not at all
flg = f'{method}_{noise}' # flg to save data

path = f'/data/vnikitin/paper/near_field' # input data path 
path_out = f'/data/vnikitin/paper/near_field/rec0' #output data path

alpha_psi = 0.1
alpha_prb = 0.1

In [ ]:

def mshow(a, show=True, **args):
    """Plot the 2D array, handling arrays on GPU      

    Parameters
    ----------
    a : (ny, nx) float32
        2D array for visualization
    args : 
        Other parameters for imshow    
    """
    if not show:
        return

    fig, axs = plt.subplots(1, 1, figsize=(3, 3))
    im = axs.imshow(a, cmap='gray', **args)
    fig.colorbar(im, fraction=0.046, pad=0.04)
    plt.show()
def mshow_complex(a, show=True, **args):
    """Plot the 2D array in the rectangular representation with the real and imag parts, 
    handling arrays on GPU   

    Parameters
    ----------
    a : (ny, nx) complex64
        2D array for visualization
    args : 
        Other parameters for imshow    
    """
    if not show:
        return
    fig, axs = plt.subplots(1, 2, figsize=(9, 3))
    im = axs[0].imshow(a.real, cmap='gray', **args)
    axs[0].set_title('real')
    fig.colorbar(im, fraction=0.046, pad=0.04)
    im = axs[1].imshow(a.imag, cmap='gray', **args)
    axs[1].set_title('imag')
    fig.colorbar(im, fraction=0.046, pad=0.04)
    plt.show()
def mshow_polar(a, show=False, **args):
    """Plot the 2D array in the polar representation with the absolute value and phase,
    handling arrays on GPU       

    Parameters
    ----------
    a : (ny, nx) complex64
        2D array for visualization
    args : 
        Other parameters for imshow    
    """
    if not show:
        return
    fig, axs = plt.subplots(1, 2, figsize=(9, 3))
    im = axs[0].imshow(np.abs(a), cmap='gray', **args)
    axs[0].set_title('abs')
    fig.colorbar(im, fraction=0.046, pad=0.04)
    im = axs[1].imshow(np.angle(a), cmap='gray', **args)
    axs[1].set_title('phase')
    fig.colorbar(im, fraction=0.046, pad=0.04)
    plt.show()

In [ ]:
shifts = cp.load(f'{path}/data/gen_shifts.npy')[:npos]
prb = cp.load(f'{path}/data/gen_prb.npy')
if noise:
    data = cp.load(f'{path}/data/ndata.npy')[:npos]
else:
    data = cp.load(f'{path}/data/data.npy')[:npos]
ref = cp.load(f'{path}/data/ref.npy')
psi = cp.load(f'{path}/data/psi.npy')

mshow_polar(prb.get(),show)
mshow_complex((data[0]+1j*data[0]/ref).get(),show,vmax=3)

In [ ]:
def Paganin(data, wavelength, voxelsize, delta_beta,  alpha):
    fx = cp.fft.fftfreq(data.shape[-1], d=voxelsize).astype('float32')
    [fx, fy] = cp.meshgrid(fx, fx)
    rad_freq = cp.fft.fft2(data)
    taylorExp = 1 + wavelength * distance * cp.pi * (delta_beta) * (fx**2+fy**2)
    numerator = taylorExp * (rad_freq)
    denominator = taylorExp**2 + alpha
    phase = cp.log(cp.real(cp.fft.ifft2(numerator / denominator)))
    phase = delta_beta * 0.5 * phase
    return phase

def rec_init(rdata,ishifts):
    recMultiPaganin = cp.zeros([nobj,nobj],dtype='float32')
    recMultiPaganinr = cp.zeros([nobj,nobj],dtype='float32')# to compensate for overlap
    for j in range(0,npos):
        r = rdata[j]        
        rr = r*0+1 # to compensate for overlap                
        rpsi = cp.ones([nobj,nobj],dtype='float32')
        rrpsi = cp.ones([nobj,nobj],dtype='float32')
        stx = nobj//2-ishifts[j,1]-n//2
        endx = stx+n
        sty = nobj//2-ishifts[j,0]-n//2
        endy = sty+n
        rpsi[sty:endy,stx:endx] = r
        rrpsi[sty:endy,stx:endx] = rr
        rpsi = Paganin(rpsi, wavelength, voxelsize,  24.05, 5e-3)
        recMultiPaganin += rpsi
        recMultiPaganinr += rrpsi
        
    recMultiPaganinr[cp.abs(recMultiPaganinr)<5e-2] = 1    
    recMultiPaganin /= recMultiPaganinr    
    recMultiPaganin = cp.exp(1j*recMultiPaganin)
    return recMultiPaganin

ishifts = cp.round(shifts).astype('int32')
rdata = data/(ref+eps)
rec_paganin = rec_init(rdata,ishifts)
mshow_polar(rec_paganin.get(),show)

In [ ]:
prb = prb.get()
rec_paganin = rec_paganin.get()
shifts = shifts.get()
data = data.get()

In [ ]:
# positions_px = -shifts
# probe = 1+0*torch.from_numpy(prb[np.newaxis,np.newaxis])

# step=0.15

# options = api.EPIEOptions()
    
# options.data_options.data = np.fft.fftshift(data,axes=(-2,-1))

# options.object_options.initial_guess = torch.ones([1, nobj,nobj], dtype=get_default_complex_dtype())
# options.object_options.pixel_size_m = voxelsize
# options.object_options.optimizable = True
# options.object_options.optimizer = api.Optimizers.SGD
# options.object_options.step_size = step
# options.object_options.alpha = 1

# options.probe_options.initial_guess = probe
# options.probe_options.optimizable = True
# options.probe_options.optimizer = api.Optimizers.SGD
# options.probe_options.step_size = step
# options.probe_options.alpha = 1

# options.probe_position_options.position_x_px = positions_px[:, 1]
# options.probe_position_options.position_y_px = positions_px[:, 0]
# options.probe_position_options.optimizable = False

# options.reconstructor_options.batch_size = 16
# options.reconstructor_options.num_epochs = 16

# options.reconstructor_options.csv = flg

# options.object_options.initial_guess = torch.from_numpy(rec_paganin[np.newaxis])
# # options.object_options.initial_guess = torch.from_numpy(psi[np.newaxis])
# task = PtychographyTask(options)
# task.run()

# recon = task.get_data_to_cpu('object', as_numpy=True)[0]
# rec_prb = task.get_data_to_cpu('probe', as_numpy=True)[0]
# rec_pos = -task.get_data_to_cpu('probe_positions', as_numpy=True)#[0]
# # print(rec_pos)
# dxchange.write_tiff(np.abs(recon),f'{path_out}_{flg}/crec_psi_abs/0',overwrite=True)
# dxchange.write_tiff(np.angle(recon),f'{path_out}_{flg}/crec_psi_angle/0',overwrite=True)
# dxchange.write_tiff(np.abs(rec_prb[0]),f'{path_out}_{flg}/crec_prb_abs/0',overwrite=True)
# dxchange.write_tiff(np.angle(rec_prb[0]),f'{path_out}_{flg}/crec_prb_angle/0',overwrite=True)
# np.save(f'{path_out}_{flg}/crec_shift_0',rec_pos)
# mshow_polar(recon,mshow)

In [ ]:
positions_px = -shifts
probe = 1+0*torch.from_numpy(prb[np.newaxis,np.newaxis])

step=1.5#if error in shifts are big 1.3

options = api.LSQMLOptions()
options.data_options.data = np.fft.fftshift(data,axes=(-2,-1))

options.object_options.initial_guess = torch.ones([1, nobj,nobj], dtype=get_default_complex_dtype())
options.object_options.pixel_size_m = voxelsize
options.object_options.optimizable = True
options.object_options.optimizer = api.Optimizers.SGD
options.object_options.step_size = step

options.probe_options.initial_guess = probe
options.probe_options.optimizable = True
options.probe_options.optimizer = api.Optimizers.SGD
options.probe_options.step_size = step

options.probe_position_options.position_x_px = positions_px[:, 1]
options.probe_position_options.position_y_px = positions_px[:, 0]
options.probe_position_options.optimizable = False

options.reconstructor_options.batch_size = 16
options.reconstructor_options.noise_model = api.NoiseModels.GAUSSIAN
options.reconstructor_options.num_epochs = 32

options.reconstructor_options.csv = flg 

options.object_options.initial_guess = torch.from_numpy(rec_paganin[np.newaxis])#rec_paganin[np.newaxis])
# options.object_options.initial_guess = torch.from_numpy(psi[np.newaxis])#rec_paganin[np.newaxis])
task = PtychographyTask(options)

task.run()

recon = task.get_data_to_cpu('object', as_numpy=True)[0]
rec_prb = task.get_data_to_cpu('probe', as_numpy=True)[0]

mshow_polar(recon,mshow)
# mshow_polar(rec_prb[0],mshow)

# os.system(f"cp lsqml lsqml_{flg}")
dxchange.write_tiff(np.abs(recon),f'{path_out}_{flg}/crec_psi_abs/0',overwrite=True)
dxchange.write_tiff(np.angle(recon),f'{path_out}_{flg}/crec_psi_angle/0',overwrite=True)
dxchange.write_tiff(np.abs(rec_prb[0]),f'{path_out}_{flg}/crec_prb_abs/0',overwrite=True)
dxchange.write_tiff(np.angle(rec_prb[0]),f'{path_out}_{flg}/crec_prb_angle/0',overwrite=True)

In [ ]:
mshow_polar(recon,mshow)